## Backprop Workbook 02: Backprop to Hidden Layer

**For these questions, assume that an $x$ input has 1024 dimensions, that the first hidden layer should have $512$ units, a second layer has $256$ units, and that there are $10$ classes to choose from at the end.**

**Cell to run for Latex commands**

\\[
\newcommand{\fpartial}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\grad}[1]{\nabla #1}
\newcommand{\softmax}[0]{\text{SOFTMAX}}
\newcommand{\trans}[1]{{#1}^{\intercal}}
\\]

## Backprop Further Through Gradient Chains

We've now calculated gradients for the final layer's weights and biases. We now want to calculate the corresponding gradients for $\grad_{W^{(2)}} CE(h^{(3)}, y)$ and $\grad_{b^{(2)}} CE(h^{(3)}, y)$.

How do changes to these weights and biases effect the loss?

1. Changes to $W^{(2)}$ and $b^{(2)}$ change the $z^{(2)}$ values.
2. Changes to the $z^{(2)}$ values change the $h^{(2)}$ values.
3. Changes to the $h^{(2)}$ values change the $z^{(3)}$ values.
4. And we already know how changes to $z^{(3)}$ values change the loss.

In this section we will work our way backward: 3, 2, 1. Blast off!

**1. Write $\grad_{b^{(2)}} CE(h^{(3)}, y)$ as a chain of four gradients. Take inspiration from the above plan.**

\\[
\grad_{b^{(2)}} CE(h^{(3)}, y)
=
\left(
    \grad_{b^{(2)}} z^{(2)}
\right)
\cdot
\left(
    \grad_{z^{(2)}} h^{(2)}
\right)
\cdot
\left(
    \grad_{h^{(2)}} z^{(3)}
\right)
\cdot
\left(
    \grad_{z^{(3)}} CE(h^{(3)}, y)
\right)
\\]

**2. Write $\grad_{W^{(2)}} CE(h^{(3)}, y)$ as a chain of four gradients. Take inspiration from the above plan.**

\\[
\grad_{W^{(2)}} CE(h^{(3)}, y)
=
\left(
    \grad_{W^{(2)}} z^{(2)}
\right)
\cdot
\left(
    \grad_{z^{(2)}} h^{(2)}
\right)
\cdot
\left(
    \grad_{h^{(2)}} z^{(3)}
\right)
\cdot
\left(
    \grad_{z^{(3)}} CE(h^{(3)}, y)
\right)
\\]

## Chain Rule For Gradients

**1. Gradients like $\grad_{z^{(3)}} CE(h^{(3)}, y)$ are familiar because there are many inputs, but only one output. What is the shape of this vector? Why?**

It is a row vector of length ten because there are ten $z^{(3)}$ values and thus ten partial derivatives. The gradient matches the shape of $z^{(3)}$.

**2. When we write $\Delta z^{(3)} \cdot \grad_{z^{(3)}} CE(h^{(3)}, y) = \Delta CE(h^{(3)}, y)$, what are the shapes of the vectors involved?**


The first two are vectors of shape $(10,)$. The last is a scalar.

**3. The next step is to try to calculate $\grad_{h^{(2)}} z^{(3)}$. I want to write $\Delta h^{(2)} \cdot \grad_{h^{(2)}} z^{(3)} = \Delta z^{(3)}$. What are the shapes involved?**

First a vector of length $(256,)$. Then a *matrix* of dimension $(256, 10)$. Finally a vector of length $(10,)$.

**4. Soon I will want to calculate $\grad_{W^{(2)}} z^{(2)}$. Whoa. Let's start with a simpler question: $\grad_{W^{(2)}} z^{(2)}_k$. What is the shape of my $\Delta W^{(2)}$?**

The shape is $(512, 256)$.

**5. What is the shape of $\grad_{W^{(2)}} z^{(2)}_k$? How is this different from $\grad_{h^{(2)}} z^{(3)}$?**

It is $(512, 256)$. The difference is that *both* dimensions are "input dimensions."

**6. How do you calculate $\Delta z^{(2)}_k$?**

You multiply corresponding entries of $\Delta W^{(2)}$ and $\grad_{W^{(2)}} z^{(2)}_k$ and sum out.

**Note:** this kind of operation is called a "double dot product." We denote it as:

\\[
\Delta W^{(2)} : \grad_{W^{(2)}} z^{(2)}_k = \Delta z^{(2)}_k
\\]

It says: take the *last* two (i.e., all) the dimensions of $W^{(2)}$ and multiply them coordinatewise with the *first* two dimensions of $\grad_{W^{(2)}} z^{(2)}_k$ (i.e., all), and then sum out.

It's just like the dot-product, but with two dimensions!

In numpy we write: `np.tensordot(A, B, axes = 2)`.

**7. Write the formula for $\Delta z^{(2)}$ now.**

\\[
\Delta W^{(2)} : \grad_{W^{(2)}} z^{(2)} = \Delta z^{(2)}
\\]

## Calculating $\grad_{h^{(2)}} z^{(3)}$ and $\grad_{h^{(2)}} CE(h^{(3)}, y)$

**1. In our plan we know the last gradient $\grad_{z^{(3)}} CE(h^{(3)}, y)$, so let's work backward and start with $\grad_{h^{(2)}} z^{(3)}$. Let's focus on just a single partial: $\fpartial{}{h^{(2)}_i} z^{(3)}_j$. Use the formula for $z^{(3)}_j$ to calculate this.**

\\[
\fpartial{}{h^{(2)}_i} z^{(3)}_j
=
\fpartial{}{h^{(2)}_i} \left(
    \sum_{k = 0}^{512} h^{(2)}_k W^{(3)}_{k, j}
\right) + b^{(3)}_j
=
W^{(3)}_{i, j}
\\]

**2. Why does does $W^{(3)}_{i, j}$ feel like the right anwer?**

Because it is the weight that connects $h^{(2)}_i$ to $z^{(3)}_j$. Any change in $h^{(2)}_i$ will be "magnified" by $W^{(3)}_{i, j}$.

**3. Using this result, and our definition above for $\left(\grad_{h^{(2)}} z^{(3)}\right)_{i, j}$, give an equation for $\grad_{h^{(2)}} z^{(3)}$.**


\\[
\grad_{h^{(2)}} z^{(3)}
=
W^{(3)}
\\]

**4. Great. Let's break this down to understand better. Give a formula for $\grad_{h^{(2)}} z^{(3)}_j$ in terms of $W^{(3)}$. What is the shape of this? Why does this formula make sense?**

\\[
\grad_{h^{(2)}} z^{(3)}_j = W^{(3)}_{:, j}
\\]

This is a vector with length $256$. It makes sense because all 256 units of $h^{(2)}$ are connected to the $z^{(3)}_j$ value. This column consists of exactly the weights used to calculate $z^{(3)}_j$ and scale the values in $h^{(2)}$.

**5. Using the above formula, consider a change $\Delta h^{(2)}$ to the 256 dimensions of $h^{(2)}$. Use the gradient for $z^{(3)}_j$ to calculate the change in $z^{(3)}_j$. Break it down to the summation level even. Do this both in terms of partials $\fpartial{z^{(3)}_j}{h^{(2)}_i}$ and $W^{(3)}$. Give an explanation for the formulae.**

\\[
\begin{align}
    \Delta z^{(3)}_j
    &=
    \left(
        \Delta h^{(2)}
    \right)
    \cdot
    \left(
        \grad_{h^{(2)}} z^{(3)}_j
    \right)
    =
    \sum_{i = 0}^{256}
    \Delta h^{(2)}_i \fpartial{z^{(3)}_j}{h^{(2)}_i}
\\
    &=
    \left(
        \Delta h^{(2)}
    \right)
    \cdot
    \left(
        W^{(2)}_{:, j}
    \right)
    =
    \sum_{i = 0}^{256}
    \Delta h^{(2)}_i W^{(3)}_{i, j}
\end{align}
\\]

Basically each change to $h^{(2)}$ has its own impact on the $z^{(3)}_j$ value. We need to evaluate those impacts and sum them up.

**6. Give a formula for $\grad_{h^{(2)}_i} z^{(3)}$ in terms of $W^{(3)}$. What is the shape of this? Column or row vector? Why? Why does this formula make sense?**

\\[
\grad_{h^{(2)}_i} z^{(3)} = W^{(3)}_{i, :}
\\]

This is a vector of length $10$. This way when multiplied by a scalar change $\Delta h^{(2)}_i$ you get $\Delta z^{(3)}$.

It makes sense because the unit $h^{(2)}_i$ is connected to all $10$ units of $z^{(3)}$.

**7. Using the above formula, consider a scalar change $\Delta h^{(2)}_i$. Calculate the change in $z^{(3)}$. Do this both in terms of partials $\fpartial{z^{(3)}_j}{h^{(2)}_i}$ and $W^{(3)}$. Give an explanation for the formulas.**

\\[
\begin{align}
    \Delta z^{(3)}
    &=
    \Delta h^{(2)}_i \grad_{h^{(2)}_i} z^{(3)}
    =
    \left(
        \Delta h^{(2)}_i
        \fpartial{z^{(3)}_0}{h^{(2)}_i}
        ,
        \Delta h^{(2)}_i
        \fpartial{z^{(3)}_1}{h^{(2)}_i}
        ,
        \ldots
        ,
        \Delta h^{(2)}_i
        \fpartial{z^{(3)}_255}{h^{(2)}_i}
    \right)
\\
    &=
    \Delta h^{(2)}_i W^{(3)}_{i, :}
    =
    \left(
        \Delta h^{(2)}_i
        W^{(3)}_{i, 0}
        ,
        \Delta h^{(2)}_i
        W^{(3)}_{i, 1}
        ,
        \ldots
        ,
        \Delta h^{(2)}_i
        W^{(3)}_{i, 255}
    \right)
\end{align}
\\]

**8. The chain rule says:**

\\[
\grad_{h^{(2)}} CE(h^{(3)}, y)
=
\left(
    \grad_{h^{(2)}} z^{(3)}
\right)
\cdot
\left(
    \grad_{z^{(3)}} CE(h^{(3)}, y)
\right)
\\]

**Tell me the shapes of the terms in the product. Tell me about the final shape.**

The shapes are $(256, 10)$ and $(10,1)$. The product is a vector $(256, 1)$.

**9. Why does it make sense that the final shape of the gradient is $(256, 1)$?**


There are $256$ units in layer two, but we're assessing their impact on a single scalar value: the loss.

**10. To calculate the product, we take the dot product of rows of $\grad_{h^{(2)}} z^{(3)}$ with $\grad_{z^{(3)}} CE(h^{(3)}, y)$. This dot product is $\left(\grad_{h^{(2)}} CE(h^{(3)}, y)\right)_i$.**

**Write a formula with a summation for this for row $i$ of $\grad_{h^{(2)}} z^{(3)}$. Do this both in terms of partials and in terms of $W^{(3)}$.**

\\[
\begin{align}
    \left(
        \grad_{h^{(2)}} CE(h^{(3)}, y)_i
    \right)_i
    &=
    \left(
        \grad_{h^{(2)}} z^{(3)}
    \right)_{i, :}
    \cdot
    \left(
        \grad_{z^{(3)}} CE(h^{(3)}, y)
    \right)
    =
    \sum_{j = 0}^{10}
    \left(
        \fpartial{z^{(3)}_j}{h^{(2)}_i}
    \right)
    \left(
        \fpartial{CE(h^{(3)}, y)}{z^{(3)}_j}
    \right)
\\
    &=
    W^{(3)}_{i, :}
    \cdot
    \left(
        \grad_{z^{(3)}} CE(h^{(3)}, y)
    \right)
    =
    \sum_{j = 0}^{10}
    W^{(3)}_{i, j}
    \left(
        \fpartial{CE(h^{(3)}, y)}{z^{(3)}_j}
    \right)
\end{align}
\\]

**11. Give me a story in words for this formula.**


A change in $h^{(2)}_i$ affects all of $z^{(3)}_j$ values via the weights in row $W_{i, :}$. And a change in a $z^{(3)}_j$ value causes a change in $CE(h^{(3)}, y)$.

The amount of change for $CE(h^{(3)}, y)$ "via" the value $z^{(3)}_j$ is equal to the product of the two partial derivatives.

The total change in cross-entropy comes from summing up over all the "routes" via which $h^{(2)}_i$ can impact the cross-entropy.

**12. Calculate $\grad_{h^{(2)}} CE(h^{(3)}, y)$ by using the formulae we've found for $\grad_{h^{(2)}} z^{(3)}$ and $\grad_{z^{(3)}} CE(h^{(3)}, y)$.**

\\[
\begin{align}
    \grad_{h^{(2)}} CE(h^{(3)}, y)
    &=
    \left(
        \grad_{h^{(2)}} z^{(3)}
    \right)
    \cdot
    \left(
        \grad_{z^{(3)}} CE(h^{(3)}, y)
    \right)
\\
    &=
    W^{(3)}
    \cdot
    (h^{(3)} - y)
\end{align}
\\]

## Calculating $\grad_{z^{(2)}} h^{(2)}$ and $\grad_{z^{(2)}} CE(h^{(3)}, y)$

Since we know 

\\[
\grad_{z^{(2)}} CE(h^{(3)}, y)
=
\left(\grad_{z^{(2)}} h^{(2)}\right)
\cdot
\left(\grad_{h^{(2)}} CE(h^{(3)}, y)\right)
\\]

We know that what we really need to backprop the loss to $z^{(2)}$ is to calculate $\grad_{z^{(2)}} h^{(2)}$.

**1. What are the shapes of $\grad_{z^{(2)}} h^{(2)}$, $\grad_{h^{(2)}} CE(h^{(3)}, y)$ and $\grad_{z^{(2)}} CE(h^{(3)} y)$.**


They are $(256, 256)$ $(256, 1)$, and $(256, 1)$.

**2. $\grad_{z^{(2)}} h^{(2)}$ is a matrix which records how a change to any $z^{(2)}_i$ can change any $h^{(2)}_j$. Why does this seem excessive? How many and which $h^{(2)}_j$ values respond to a change in $z^{(2)}_i$? Why? Consider the formula for $h^{(2)}_j$ in terms of the $z^{(2)}$ values...**

We know:

\\[
h^{(2)}_j = \sigma\left(z^{(2}_j\right)
\\]

Therefore, the only $z^{(2)}$ value that can effect $h^{(2)}_j$ is $z^{(2)}_j$. The other $z^{(2)}$ values have no impact at all on $h^{(2)}_j$.

Our gradient matrix $\grad_{z^{(2)}} h^{(2)}$ will hold almost all zeros!

**3. What entries of the gradient matrix $\grad_{z^{(2)}} h^{(2)}$ will be zero, and which can be non-zero? What do we call this kind of matrix? What are the values of these entries in terms of partials?**

Only at positions $(i, j)$ where $i = j$ can the matrix be non-zero. All other entries must be zero.

We have:

\\[
\left(
     \grad_{z^{(2)}} h^{(2)}
\right)_{i, i}
=
\fpartial{h^{(2)}_i}{z^{(2)}_i}
\\]

This is called a *diagonal* matrix.

**4. What does a diagonal matrix do to a vector? What kind of operation is it? Think about what a $3$-by-$3$ diagonal matrix does to $(1, 0, 0)$, $(0, 1, 0)$, and $(0, 0, 1)$.**

The digonal matrix stretches unit vectors.

**5. What does a diagonal matrix $D$ do to a matrix $M$? Think about composition of operations when considering $DM$: think about $(xD)M$. Why?**

The matrix $D$ stretches each row of $M$. That's because any input $xD$ to $M$ will have been "pre-stretched." So we can just fold the stretching into $M$ by multiplying appropriate rows.

**6. We must now learn how to calculate the partial $\fpartial{\sigma(z)}{z}$. The first step is to learn a formula for $1 - \sigma(z)$. Calculate this by expanding the definition of $\sigma$ and simplifying. Hint: remember both formulas for $\sigma(z)$...**

\\[
1 - \sigma(z)
=
1 - \frac{e^z}{1 + e^z}
=
\frac{1 + e^z}{1 + e^z} - \frac{e^z}{1+e^z}
=
\frac{1}{1 + e^z}
\\]

That's almost the same as the "other" formula for $\sigma(z)$. Except the other formula has a $-z$ in the denominator. So this is actually the formula for $\sigma(-z)$.

Thus we have 

\\[
1 - \sigma(z)
=
\sigma(-z)
\\]

**7. Why does this formula make sense? Think about the relation of an odds $A:B$ vs odds of $B:A$.**

If $A:B$ is three-to-one odds, then $B:A$ is one-to-three odds. That is, the relationship is $odds_{B:A} = \left(odds_{A:B}\right)^{-1}$. In the log scale, that turns out to $\log odds_{B:A} = -\log odds_{A:B}$.

**8. Next, let's take the derivative of $\sigma(z)$ wrt $z$. Remember that we can write $\sigma(z) = \left(1 + e^{-z}\right)^{-1}$. This lets us use the "polynomial rule" and chain rule together.**

\\[
\begin{align}
\fpartial{\sigma(z)}{z}
&=
\fpartial{}{z}
\left(1 + e^{-z}\right)^{-1}
\\
&=
-1
\left(1 + e^{-z}\right)^{-2}
\fpartial{}{z}
e^{-z}
\\
&=
-1
\left(1 + e^{-z}\right)^{-2}
\left(
    -e^{-z}
\right)
\\
&=
\frac{e^{-z}}{\left(1 + e^{-z}\right)^2}
\\
&=
\frac{1}{\left(1 + e^{-z}\right)}
\frac{e^{-z}}{\left(1 + e^{-z}\right)}
\\
&=
\sigma(z)
\frac{1}{\left(1 + e^{z}\right)}
\\
&=
\sigma(z)
\sigma(-z)
\\
&=
\sigma(z)
\left(1 - \sigma(z)\right)
\end{align}
\\]

**9. Consider our diagonal matrix $\grad_{z^{(2)}} h^{(2)}$ with $\sigma(z^{(2)}_i)(1 - \sigma(z^{(2)}_i)$ along the diagonal. We know that:**

\\[
\grad_{z^{(2)}} CE(h^{(3)}, y)
=
\left(\grad_{z^{(2)}} h^{(2)}\right)
\cdot
\left(\grad_{h^{(2)}} CE(h^{(3)}, y)\right)
\\]

**When we take this product, what happens to each element of $\grad_{h^{(2)}} CE(h^{(3)}, y)$? Explain.**


Each element $\left(\grad_{h^{(2)}} CE(h^{(3)}, y)\right)_i$ is scaled by $\sigma(z^{(2)}_i)(1 - \sigma(z^{(2)}_i))$.

The reason is that the impact of $z^{(2)}_i$ on $h^{(2)}_i$ is $\sigma(z^{(2)}_i)(1 - \sigma(z^{(2)}_i))$. And then the impact of $h^{(2)}_i$ on the loss is $\left(\grad_{h^{(2)}} CE(h^{(3)}, y)\right)_i$.

So they multiply together.

**10. This sounds like a job for the Hadamard AKA coordinatewise product $\odot$. Write a formula for $\grad_{z^{(2)}} CE(h^{(3)}, y)$.**

\\[
\grad_{z^{(2)}} CE(h^{(3)}, y)
=
\sigma(z) \odot (1 - \sigma(z))
\odot
\grad_{h^{(2)}} CE(h^{(3)}, y)
\\]

## Backprop to $\grad_{b^{(2)}} CE(h^{(3)}, y)$

**1. What is the shape of $\grad_{b^{(2)}} z^{(2)}$?**

$(256, 256)$.

**2. What is $\fpartial{z^{(2)}_j}{b^{(2)}_j}$? Why?**


Since $z^{(2)}_j = b^{(2)}_j + \sum_{i=0}^{512} h^{(1)}_i W^{(2)}_{i, j}$, the partial is $1$.

**3. Why does $\grad_{b^{(2)}} z^{(2)}$ equal the identity matrix (ones along the diagonal, zeros everywhere else?**

By the above equation all the diagonal entries equal one. Also the non-diagonal entries must all be zero because no $b^{(2)}_j$ affects any other $z^{(2)}_k$.

**4. For an identity matrix $I$, what is $IM$? Why?**

It is $M$, by the above reasoning about the action of diagonal matrices as unit-vector stretchers. The identity matrix doesn't do anything to $xI$. So $xIM = xM$.

**5. Therefore, what is $\grad_{b^{(2)}} CE(h^{(3)}, y)$?**

It is $\grad_{z^{(2)}} CE(h^{(3)}, y)$.

## Backprop to $\grad_{W^{(2)}} CE(h^{(3)}, y)$ using $\grad_{z^{(2)}} CE(h^{(3)}, y)$

**1. Consider $\grad_{W^{(2)}} CE(h^{(3)}, y)$. What is its shape?**

It is $(512, 256)$.


**2. Consider a single entry $W^{(2)}_{i, j}$. We want to calculate $\fpartial{}{W^{(2)}_{i, j}} CE(h^{(3)}, y)$. Let's first calculate $\fpartial{}{W^{(2)}_{i, j}} z^{(2)}_j$. Explain the formula.**

Since

\\[
z^{(2)}_j = b^{(2)}_j + \sum_{i = 0}^{512} h^{(1)}_i W^{(2)}_{i, j}
\\]

we have that the partial is $h^{(1)}_i$. That's because this weight connects $h^{(1)}_i$ to $z^{(2)}_j$.

**3. Consider $\fpartial{}{W^{(2)}_{i, j}} z^{(2)}_k$ for any $k\ne j$. What is this? Why?**

It is zero because this weight does not contribute to $z^{(2)}_k$.

**4. What is $\grad_{W^{(2)}_{i, j}} z^{(2)}$ then?**

It is all zeros, except for $h^{(1)}_i$ at position $j$.

**5. Using the chain rule for $\grad_{W^{(2)}_{i, j}} CE(h^{(3)}, y) = \grad_{W^{(2)}_{i, j}} z^{(2)} \cdot \grad_{z^{(2)}} CE(h^{(3)}, y)$, what do you compute?**

We get $h^{(1)}_i \left(\grad_{z^{(2)}} CE(h^{(3)}, y))\right)_j$.

**6. So every entry $(i, j)$ of $\grad_{W^{(2)}} CE(h^{(3)}, y)$ consists of two factors. Which factor is constant across *rows*? Explain.**

$h^{(1)}_i$. The reason is that all the weights in row $i$ connect the hidden unit $h^{(1)}_i$ to various units of $z^{(2)}$.

**7. What factor is constant across *columns*? Explain.**

$\left(\grad_{z^{(2)}} CE(h^{(3)}, y))\right)_j$. The reason is that every weight in column $j$ connects to $z^{(2)}_j$, and the change in this unit is captured by this gradient.

**8. Therefore write $\grad_{W^{(2)}} CE(h^{(3)}, y)$ as an outer product.**

$h^{(1)} \otimes \grad_{z^{(2)}} CE(h^{(3)}, y))$.

## Further Backprop

**1. At this point, you just repeat the process. You can calculate $\grad_{h^{(1)}} CE(h^{(3)}, y)$ as before. What dot product do you use?**


\\[
\grad_{h^{(1)}} CE(h^{(3)}, y)
=
W^{(2)}
\cdot
\grad_{z^{(2)}} CE(h^{(3)}, y)
\\]

**2. Likewise $\grad_{z^{(1)}} CE(z^{(3)}, y)$. What Hadamard product do you do?**

\\[
\grad_{z^{(1)}} CE(h^{(3)}, y)
=
\left(
    \sigma(z^{(1)})
    \odot
    (1 - \sigma(z^{(2)}))
\right)
\odot
\grad_{h^{(1)}} CE(h^{(3)}, y)
\\]

**3. We know that the prior gradient is equal to $\grad_{b^{(1)}} CE(z^{(3)}, y)$. What outer product do you need for $\grad_{W^{(1)}} CE(z^{(3)}, y)$?**

\\[
\grad_{W^{(1)}} CE(z^{(3)}, y)
=
x
\otimes
\grad_{z^{(1)}} CE(h^{(3)}, y)
\\]